In [1]:
import rootpath
import os
import sys
sys.path.append(rootpath.detect())

import json 
import numpy as np
import wfg
import time
import matplotlib.pyplot as plt

from IPython.display import Markdown as md
from testsuite.analysis_tools import strip_problem_names, draw_samples, get_igd_refpoint_dict, get_target_dict, dominates, attainment_sample, find_neighbours
from testsuite.utilities import PROBLEM_CONFIGURATIONS

%matplotlib qt

In [2]:
def generate_nonuniform_samples(name, n_samples):
    n_prob, n_obj, n_dim = strip_problem_names(name)
    problem = getattr(wfg, f"WFG{n_prob}") 
    return draw_samples(problem, n_obj, n_dim, n_samples, False, True)

def target_from_dict(D, t):
    try:
        return D[t]
    except KeyError:
        prob, obj, dim = strip_problem_names(t)
        return D[f'ellipsoid_{obj}obj']
        

def plot_2d(samples_y, uniform_y):
    fig = plt.figure(figsize=[8,8])
    ax = fig.gca()
    ax.scatter(*samples_y.T, c="C0", s=15, alpha=0.2)
    ax.scatter(*uniform_y.T, c="C3", s=2, alpha=1.0)
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    return fig

def plot_3d(samples_y, uniform_y):
    fig = plt.figure(figsize=[8,8])
    ax = fig.gca(projection="3d")
    ax.scatter(*samples_y.T, c="C0", s=15, alpha=0.2)
    ax.scatter(*uniform_y.T, c="C3", s=2, alpha=1.0)
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_zlabel("z")
    return fig

def plot_4d(samples_y, uniform_y):
    fig = plt.figure(figsize=[8,8])
    ax = fig.gca(projection="3d")
    ax.scatter(*samples_y[:, :3].T, c="C0", s=8, alpha=0.3)
    ax.scatter(*uniform_y[:, :3].T, c="C3", s=2, alpha=1.0)
    return fig
    

def plot_igd_points(problem_name):
    
    prob, obj, dim = strip_problem_names(problem_name)
    prob = getattr(wfg, f"WFG{prob}")
    print(problem_name, f"\t:\t function {prob}, obj {obj}, dims {dim}")
    
    print(f"drawing {N_POINTS[obj]} random samples.")
    samples_x, samples_y = draw_samples(func=prob, n_obj=obj, n_dim=dim, n_samples=N_POINTS[obj], random=False)
    
    uniform_y = np.asarray(refpoint_dict[problem_name])
    
    if obj == 2:
        fig = plot_2d(samples_y, uniform_y)
    elif obj == 3:
        fig = plot_3d(samples_y, uniform_y)
    elif obj == 4:
        fig = plot_4d(samples_y, uniform_y)
    return fig

In [3]:
D_igd = get_igd_refpoint_dict()
D_targets = get_target_dict()

steps:

    - Draw non-uniform samples: many
    
    - find those which dominate target 
    
    - attainment sample these
    
    - attainment sample diff

In [4]:
problem = PROBLEM_CONFIGURATIONS[0]
targets = D_targets[problem]
problem

'wfg1_2obj_3dim'

In [5]:
problem = "wfg4_3obj_4dim"

In [6]:
targets = D_targets["wfg4_3obj_8dim"]

In [7]:
P = generate_nonuniform_samples(problem, 1000000)[1]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000000/1000000 [00:06<00:00, 144745.52it/s]


In [8]:
"""
to check for which of b by are dominated by ANY of a do:
    ans = dominates(a, b)
"""

'\nto check for which of b by are dominated by ANY of a do:\n    ans = dominates(a, b)\n'

In [9]:
tic = time.time()
print(time.time()-tic)

8.606910705566406e-05


In [10]:
tic = time.time()
print(time.time()-tic)

8.916854858398438e-05


In [17]:
from scipy.spatial import distance_matrix
from scipy.ndimage.filters import gaussian_filter
def find_selection_threshold(P ,target, show_fig=True):
    
    # down_sample P to only those which dominate/are dominated by targets
    P_dom_inds = [dominates(target, ai) for ai in P]
    P_dom = P[P_dom_inds]
    
    # attainment sample relevant region
    A = attainment_sample(P_dom, 500) 
    
    A_max = A.max(axis=0)
    A_min = A.min(axis=0)
    
#     scope = np.logical_and(np.all(P_dom>A_min, axis=1), np.all(P_dom<A_max, axis=1))
    
    M = distance_matrix(A, P)
    min_distances = M.min(axis=1)
    
    fig = plt.figure()
    ax = fig.gca(projection="3d")
#     ax.scatter(*P_dom.T, c="C0", s=5, alpha=0.2)
    ax.scatter(*P[:1000].T, c="C0", s=5, alpha=0.2)
#     ax.scatter(*A.T, c="C3", s=5)
    
    fig2 = plt.figure()
    ax2 = fig2.gca()
    hist_counts = ax2.hist(min_distances, 100)
    counts, values, _ = hist_counts
    
    grads = np.abs(counts[1:]-counts[:-1])
    ax2.plot(values[1:-1], grads, c="C1")
    
    g_locs = np.where(grads<0.1)[0]
    thresh = values[g_locs[1]]
    
    ax2.axvline(thresh, c="C3", linestyle="--")
    
    A_ = A[min_distances<thresh]
    ax.scatter(*A_.T, c="C3", s=2)
    plt.show()
    return A_
    
    
A_, A_f = find_selection_threshold(P, targets[0])

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:09<00:00, 52.13it/s]
/tmp/ipykernel_733726/3785946284.py:21: MatplotlibDeprecationWarning: Calling gca() with keyword arguments was deprecated in Matplotlib 3.4. Starting two minor releases later, gca() will take no keyword arguments. The gca() function should only be used to get the current axes, or if no axes exist, create new axes with default keyword arguments. To create a new axes with non-default arguments, use plt.axes() or plt.subplot().
  ax = fig.gca(projection="3d")


ValueError: too many values to unpack (expected 2)

In [233]:
fig = plt.figure()
ax = fig.gca(projection="3d")
ax.scatter(*A_.T, c="C3", s=2, alpha=0.35)
ax.scatter(*A_f.T, c="C0", s=2, alpha=0.35)
ax.scatter(*targets[0], c="magenta")

/tmp/ipykernel_706838/1931998769.py:2: MatplotlibDeprecationWarning: Calling gca() with keyword arguments was deprecated in Matplotlib 3.4. Starting two minor releases later, gca() will take no keyword arguments. The gca() function should only be used to get the current axes, or if no axes exist, create new axes with default keyword arguments. To create a new axes with non-default arguments, use plt.axes() or plt.subplot().
  ax = fig.gca(projection="3d")
